** Я использовал 2 подхода**

1. **Первый подход: обогащение выборки c помощью Negative Sampling**  
   В этом методе мы взяли исходные пары «контекст → следующий трек» и дополнили их случайными негативными примерами, пометив их время прослушивания как 0.  
   Для каждого контекста удаляли реальные положительные треки и из оставшегося множества выбирали до 10 случайных треков, что увеличивало обучающую выборку на ~50 000 записей.  
   Полученные пары (контекст, негативный трек, 0.0) объединялись с исходными, и на этом датасете обучались новые эмбеддинги для треков и контекстов.  
   Попытки кластеризации эмбеддингов и отбора негативных треков из разных кластеров тоже не дали устойчивого прироста качества.

2. **Второй подход: конкатенация эмбеддингов артистов и треков**  
   Здесь к эмбеддингам контекста и кандидата добавляются эмбеддинги соответствующих артистов, что позволяет модели учитывать предпочтения не только по трекам, но и по исполнителям.  
   В архитектуре `ContextualRec` четыре эмбеддинга (контекст-трек, кандидат-трек, контекст-артист, кандидат-артист) объединяются и пропускаются через два полносвязных слоя с активацией ReLU.  
   Такая модификация позволила модели лучше различать ситуации, когда слушатель привязан к артисту, а не к конкретному треку.  
   Данный подход оказался более эффективным и лёгко масштабируемым за счёт небольшой добавочной размерности артиста.  

In [1]:
!pip install pytorch_lightning


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 40.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [2]:
import os
import json
import time
import yaml
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

In [6]:
with open('data.json','r') as f:
    interactions = pd.read_json(f, lines=True)
with open('tracks.json','r') as f:
    track_meta = pd.read_json(f, lines=True)

In [7]:
from collections import namedtuple
Pair = namedtuple('Pair',['user','start','track','time','latency','recommendation','experiments'])
def get_pairs(df_user):
    lst=[]
    first=None
    for _,r in df_user.sort_values('timestamp').iterrows():
        if first is None:
            first=r['track']
        else:
            lst.append((r['user'], first, r['track'], r['time']))
        if r['message']=='last':
            first=None
    return pd.DataFrame(lst, columns=['user','start','track','time'])

pairs = interactions.groupby('user').apply(get_pairs).reset_index(drop=True)

<ipython-input-7-48c16da22098>:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pairs = interactions.groupby('user').apply(get_pairs).reset_index(drop=True)
<ipython-input-7-48c16da22098>:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pairs = interactions.groupby('user').apply(get_pairs).reset_index(drop=True)


In [8]:
encoder_track = LabelEncoder().fit(track_meta.track)
encoder_artist = LabelEncoder().fit(track_meta.artist)

pairs['start_idx'] = encoder_track.transform(pairs.start)
pairs['track_idx'] = encoder_track.transform(pairs.track)
track_meta['track_idx'] = encoder_track.transform(track_meta.track)
track_meta['artist_idx'] = encoder_artist.transform(track_meta.artist)

pairs = pairs.merge(track_meta[['track_idx','artist_idx']].rename(
    columns={'track_idx':'start_idx','artist_idx':'artist_start_idx'}), on='start_idx')
# track artist:
track_meta = track_meta.rename(columns={'artist_idx':'artist_track_idx'})
pairs = pairs.merge(track_meta[['track_idx','artist_track_idx']].rename(
    columns={'track_idx':'track_idx'}), on='track_idx')


In [9]:
class ContextualRec(nn.Module):
    def __init__(self, n_tracks, n_artists, emb_dim=64, art_emb_dim=32):
        super().__init__()
        self.track_emb = nn.Embedding(n_tracks, emb_dim)
        self.artist_emb = nn.Embedding(n_artists, art_emb_dim)
        self.fc = nn.Sequential(
            nn.Linear(emb_dim*2 + art_emb_dim*2, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
    def forward(self, ctx, tr, art_ctx, art_tr):
        e1=self.track_emb(ctx); e2=self.track_emb(tr)
        a1=self.artist_emb(art_ctx); a2=self.artist_emb(art_tr)
        x=torch.cat([e1,e2,a1,a2],dim=1)
        return self.fc(x).squeeze()


In [10]:
class PairDataset(Dataset):
    def __init__(self, df):
        self.ctx = torch.LongTensor(df.start_idx.values)
        self.tr = torch.LongTensor(df.track_idx.values)
        self.ac = torch.LongTensor(df.artist_start_idx.values)
        self.at = torch.LongTensor(df.artist_track_idx.values)
        self.y = torch.FloatTensor(df.time.values)
    def __len__(self): return len(self.y)
    def __getitem__(self,i): return self.ctx[i],self.tr[i],self.ac[i],self.at[i],self.y[i]

mask = np.random.rand(len(pairs))
train_df = pairs[mask<0.8]; val_df = pairs[(mask>=0.8)&(mask<0.9)]; test_df = pairs[mask>=0.9]
train_loader = DataLoader(PairDataset(train_df), batch_size=1024, shuffle=True)
val_loader = DataLoader(PairDataset(val_df), batch_size=1024)
test_loader = DataLoader(PairDataset(test_df), batch_size=1024)

In [11]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ContextualRec(
    n_tracks=len(encoder_track.classes_),
    n_artists=len(encoder_artist.classes_)
).to(DEVICE)
opt = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()
for epoch in range(10):
    model.train(); total=0
    for ctx,tr,ac,at,y in train_loader:
        ctx,tr,ac,at,y=[x.to(DEVICE) for x in (ctx,tr,ac,at,y)]
        opt.zero_grad(); pred=model(ctx,tr,ac,at)
        loss=loss_fn(pred,y); loss.backward(); opt.step()
        total+=loss.item()
    print(f"Epoch {epoch+1} loss {total/len(train_loader):.4f}")

Epoch 1 loss 0.1635
Epoch 2 loss 0.1418
Epoch 3 loss 0.1343
Epoch 4 loss 0.1278
Epoch 5 loss 0.1219
Epoch 6 loss 0.1147
Epoch 7 loss 0.1071
Epoch 8 loss 0.0994
Epoch 9 loss 0.0911
Epoch 10 loss 0.0826


In [12]:
output_file = 'recs_ab.json'
with open(output_file,'w') as fout:
    for _,row in test_df.iterrows():
        ctx = torch.LongTensor([row.start_idx]).to(DEVICE)
        art_ctx = torch.LongTensor([row.artist_start_idx]).to(DEVICE)
        all_tr = torch.arange(len(encoder_track.classes_)).to(DEVICE)
        all_art = torch.LongTensor(track_meta.artist_track_idx.values).to(DEVICE)
        start_time = time.time()
        with torch.no_grad():
            scores = model(ctx.repeat(len(all_tr)), all_tr, art_ctx.repeat(len(all_tr)), all_art)
        latency = time.time() - start_time
        rec = encoder_track.inverse_transform([scores.argmax().cpu().item()])[0]
        entry = {
            "user": int(row.user),
            "track": int(row.track),
            "time": float(row.time),
            "latency": float(latency),
            "recommendation": int(rec),
            "experiments": {"ALL": "T4"}
        }
        fout.write(json.dumps(entry) + "\n")
print(f"Saved {output_file}")

Saved recs_ab.json
